<a href="https://colab.research.google.com/github/riddhikaa/detection/blob/main/text_det_mdoel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 905.3/905.3 kB 15.6 MB/s eta 0:00:00


In [ ]:
!pip install torch torchvision onnx onnxruntime albumentations

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.0 MB/s eta 0:00:00


In [ ]:
import cv2
import torch
from ultralytics import YOLO
import gradio as gr
import numpy as np

class FastDetector:
    def __init__(self, model_path, max_size=800):  # Reduced from 1024 to 800
        self.max_size = max_size
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Optimize CUDA settings
        if self.device.type == 'cuda':
            torch.cuda.empty_cache()
            torch.backends.cudnn.benchmark = True
            torch.backends.cudnn.deterministic = False
            torch.backends.cuda.matmul.allow_tf32 = True
            torch.backends.cudnn.allow_tf32 = True

        # Load and optimize model
        self.model = YOLO(model_path)
        self.model.to(self.device)

        if self.device.type == 'cuda':
            # Convert model to half precision (FP16)
            self.model = self.model.half()
            self.model.fuse()

            # Warmup with half precision
            dummy_input = torch.zeros((1, 3, max_size, max_size),
                                   device=self.device,
                                   dtype=torch.float16)
            with torch.no_grad():
                self.model(dummy_input)
            torch.cuda.synchronize()

        # Initialize cache with fixed size
        self.cache_size = 15  # reduced cache size to 15
        self.image_cache = {}

    def preprocess_image(self, image):
        """Optimized image preprocessing"""
        # Convert to RGB only if needed
        if len(image.shape) == 2:
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        elif image.shape[2] == 4:
            image = cv2.cvtColor(image, cv2.COLOR_BGRA2RGB)

        # Resize if needed
        h, w = image.shape[:2]
        scale = min(self.max_size / h, self.max_size / w)

        if scale < 1:
            new_h, new_w = int(h * scale), int(w * scale)
            image = cv2.resize(image, (new_w, new_h),
                             interpolation=cv2.INTER_AREA)

        return image, scale

    def detect(self, image_path):
        # Quick cache check
        if image_path in self.image_cache:
            return self.image_cache[image_path]

        # Read image efficiently
        image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        if image is None:
            return None

        # Process image
        original_image = image.copy()
        processed_image, scale = self.preprocess_image(image)

        # Run optimized inference
        with torch.no_grad(), torch.cuda.amp.autocast() if self.device.type == 'cuda' else nullcontext():
            results = self.model.predict(
                source=processed_image,
                conf=0.6,  # increased confidence threshold
                iou=0.4,   # reduced IOU threshold
                max_det=30,  # max detections reduced to 30
                device=self.device,
                half=True if self.device.type == 'cuda' else False,
            )

        # Process detections efficiently
        boxes = results[0].boxes
        if len(boxes) > 0:
            # Batch process coordinates
            coords = boxes.xyxy.cpu().numpy()
            if scale < 1:
                coords /= scale

            classes = boxes.cls.cpu().numpy()
            confs = boxes.conf.cpu().numpy()

            # Draw all boxes
            for coord, cls_id, conf in zip(coords, classes, confs):
                x1, y1, x2, y2 = map(int, coord)
                cls = list(ENTITIES_COLORS)[int(cls_id)]

                # Efficient drawing
                cv2.rectangle(original_image, (x1, y1), (x2, y2),
                            ENTITIES_COLORS[cls], 2)

                # Optimize text rendering
                text = f"{cls} {conf:.2f}"
                cv2.putText(original_image, text, (x1, y1-5),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                           ENTITIES_COLORS[cls], 2)

        # Update cache with size limit
        if len(self.image_cache) >= self.cache_size:
            self.image_cache.pop(next(iter(self.image_cache)))
        self.image_cache[image_path] = original_image

        return original_image

class nullcontext:
    def __enter__(self): return None
    def __exit__(self, *args): return None

# Initialize detector with smaller max size
detector = FastDetector('dla-model (1).pt', max_size=800)

# Create Gradio interface
iface = gr.Interface(
    fn=detector.detect,
    inputs=gr.Image(label="Upload scanned document", type="filepath"),
    outputs="image",
)

if __name__ == "__main__":
    iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://356ac099a66fe92699.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
